# Objected Oriented Programming Project-2

## Israa Atike (D00262160)

1- Data scraping from the internet:

The website I decided to scrape is called Morgan Mcinley, it is an irish website that presents jobs all over Ireland. 

The below code will scrape the job titles, the locations, the job types, the salaries, and the date when the job was posted, I could not scrape the company name, since the website does not provide that, since it is an agency that you can use its wesbite to apply directly to jobs.

In the below code I used the library selenuim, since the website was hard to scrape, the only solution was to use selenuim so that I can scrape the data from a virtual browser.

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd

class JobScraper:
    def __init__(self):
        self.job_titles = []
        self.locations = []
        self.job_types = []
        self.salaries = []
        self.posted_dates = []
    
    def scrape_jobs(self, num_pages=50):
        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service)
        
        for i in range(1, num_pages+1):
            url = f'https://www.morganmckinley.com/ie/jobs?keyword=Data%20Science&country=Ireland&location=&industry=&type=&lang=en&salary=&minsalary=&maxsalary=&category=&uuid=&sub_discipline_name=&qtime=1706382800&page={i}'
            driver.get(url)

            page = driver.page_source
            soup = BeautifulSoup(page, 'html.parser')

            self.job_titles.extend([h5.text.strip() for h5 in soup.find_all('h5')])

            for job_listing in soup.find_all('div', class_='job-list-summary'):
                location = job_listing.find('li', class_='lj-location').text.strip() 
                job_type = job_listing.find('li', class_='lj-type').text.strip()
                salary = job_listing.find('li', class_='lj-salary').text.strip()
                posted_date = job_listing.find('li', class_='lj-posted-on').text.strip()

                self.locations.append(location)
                self.job_types.append(job_type)
                self.salaries.append(salary)
                self.posted_dates.append(posted_date)

        
        driver.quit()
        print("Scraping complete.")
    
    def create_dataframe(self):
        jobs_data = {
            'Job Title': self.job_titles,
            'Location': self.locations,
            'Job Type': self.job_types,
            'Salary': self.salaries,
            'Posted Date': self.posted_dates
        }
        df = pd.DataFrame(jobs_data)
        print("DataFrame created.")
        return df


job_scraper = JobScraper()
job_scraper.scrape_jobs()
df = job_scraper.create_dataframe()
print(df)


Scraping complete.
DataFrame created.
                                             Job Title      Location  \
0     Data Engineer - Analyst (12 month contract Cork)     Cork City   
1                                         Data Analyst          Cork   
2                                         Data Analyst          Cork   
3                                   Data Administrator          Cork   
4                                   Data Administrator          Cork   
..                                                 ...           ...   
292                                HR Business Partner  Dublin South   
293  Vendor Management Associate- High Escalation R...        Dublin   
294                                       PPM PMO Lead        Dublin   
295                      Deposit and Insurance Advisor     Blackrock   
296                      Strategic Key Account Manager        Dublin   

      Job Type        Salary   Posted Date  
0    Permanent   Competitive  Jan 05, 2024  
1    Pe

2- Data cleaning:

In the below code I added a column to my dataframe called company, and I named all rows Morgan McKinley, since it is the agency that provides all the jobs.

In [2]:
import pandas as pd

df['Company Name'] = 'Morgan McKinley'
print(df)

                                             Job Title      Location  \
0     Data Engineer - Analyst (12 month contract Cork)     Cork City   
1                                         Data Analyst          Cork   
2                                         Data Analyst          Cork   
3                                   Data Administrator          Cork   
4                                   Data Administrator          Cork   
..                                                 ...           ...   
292                                HR Business Partner  Dublin South   
293  Vendor Management Associate- High Escalation R...        Dublin   
294                                       PPM PMO Lead        Dublin   
295                      Deposit and Insurance Advisor     Blackrock   
296                      Strategic Key Account Manager        Dublin   

      Job Type        Salary   Posted Date     Company Name  
0    Permanent   Competitive  Jan 05, 2024  Morgan McKinley  
1    Perman

Type of each column:

In the below code, I tried to find the type of each column, all of them seem to be right expect the date one that should not be a string and we should turn it into a date and time instead, for the salary I will leave that as string since for most jobs they do not specify the salary the only specification is competitive, and when they do specifiy it they give that as a range, thus I will leave it as a string.

In [3]:
variable = df.keys()
for var in variable:
    print(var,type(df[var][0]))
    

Job Title <class 'str'>
Location <class 'str'>
Job Type <class 'str'>
Salary <class 'str'>
Posted Date <class 'str'>
Company Name <class 'str'>


Checking missing data:

In the below code, the I tried to check if there is any missing data in the columns, so that I clean that, but my dataframe did not have any missing data.

In [4]:
total_miss = df.isnull().sum()
percent_miss = (total_miss/df.isnull().count()*100)
missing_data = pd.DataFrame({'Total missing':total_miss,'% missing':percent_miss})
missing_data.sort_values(by='Total missing',ascending=False).head()
print(missing_data)

              Total missing  % missing
Job Title                 0        0.0
Location                  0        0.0
Job Type                  0        0.0
Salary                    0        0.0
Posted Date               0        0.0
Company Name              0        0.0


The below code will turn the posted date column from a string to a date time class.

In [5]:
from datetime import datetime

df['Posted Date'] = [datetime.strptime(date_str, '%b %d, %Y') for date_str in df['Posted Date']]

df = pd.DataFrame(df)

print(df)



                                             Job Title      Location  \
0     Data Engineer - Analyst (12 month contract Cork)     Cork City   
1                                         Data Analyst          Cork   
2                                         Data Analyst          Cork   
3                                   Data Administrator          Cork   
4                                   Data Administrator          Cork   
..                                                 ...           ...   
292                                HR Business Partner  Dublin South   
293  Vendor Management Associate- High Escalation R...        Dublin   
294                                       PPM PMO Lead        Dublin   
295                      Deposit and Insurance Advisor     Blackrock   
296                      Strategic Key Account Manager        Dublin   

      Job Type        Salary Posted Date     Company Name  
0    Permanent   Competitive  2024-01-05  Morgan McKinley  
1    Permanent 

3- Storing the data into a database:

The below code will create a database called israa_project2_oop.

In [45]:
import mysql.connector as mysql

try:
    conn = mysql.connect(host="localhost", user="root", password="")
    cursor = conn.cursor()

    cursor.execute("CREATE DATABASE IF NOT EXISTS israa_project2_oop")

    print("Database 'israa_project2_oop' created successfully.")

except mysql.Error as e:
    print("Error %d: %s" % (e.args[0], e.args[1]))
finally:
    if 'conn' in locals() or 'conn' in globals():
        conn.close()

Database 'israa_project2_oop' created successfully.


The below code will create the table called jobs and populate it with the data that we scraped.

After normalising my data I ended up with 3 tables jobs, companies and job_types since for the column company we have the same company that is repeating, and for job type we have 3 types of jobs repeating, which are Permanent, Temporary, and contract.

In [46]:
import mysql.connector as mysql
from sqlalchemy import create_engine
import pandas as pd

try:
    conn = mysql.connect(host="localhost", user="root", password="", database="israa_project2_oop")
    cursor = conn.cursor()

    cursor.execute("""
    CREATE TABLE IF NOT EXISTS companies (
        id INT AUTO_INCREMENT PRIMARY KEY,
        name VARCHAR(255)
    )
    """)

    cursor.execute("""
    CREATE TABLE IF NOT EXISTS job_types (
        id INT AUTO_INCREMENT PRIMARY KEY,
        type VARCHAR(50)
    )
    """)

    cursor.execute("""
    CREATE TABLE IF NOT EXISTS jobs (
        id INT AUTO_INCREMENT PRIMARY KEY,
        job_title VARCHAR(255),
        location VARCHAR(255),
        salary VARCHAR(50),
        posted_date DATE,
        company_id INT,
        job_type_id INT,
        FOREIGN KEY (company_id) REFERENCES companies(id),
        FOREIGN KEY (job_type_id) REFERENCES job_types(id)
    )
    """)

    cursor.execute("INSERT INTO companies (id, name) VALUES (666, 'Morgan McKinley')")

    job_types_mapping = {
        'Permanent': 777,
        'Temporary': 999,
        'Contract': 888
    }

    for job_type, job_type_id in job_types_mapping.items():
        cursor.execute("INSERT INTO job_types (id, type) VALUES (%s, %s)", (job_type_id, job_type))


    for _, row in df.iterrows():
        posted_date_str = row['Posted Date'].strftime('%Y-%m-%d')
        
        company_id = 666

        job_type_id = job_types_mapping.get(row['Job Type'])

        cursor.execute("""
        INSERT INTO jobs (job_title, location, salary, posted_date, company_id, job_type_id) 
        VALUES (%s, %s, %s, %s, %s, %s)
        """, (row['Job Title'], row['Location'], row['Salary'], posted_date_str, company_id, job_type_id))

    conn.commit()

    cursor.execute("""
    SELECT jobs.*, companies.name AS company_name, job_types.type AS job_type
    FROM jobs
    JOIN companies ON jobs.company_id = companies.id
    JOIN job_types ON jobs.job_type_id = job_types.id
    """)
    for row in cursor.fetchall():
        print(row)

except mysql.Error as e:
    print("Error %d: %s" % (e.args[0], e.args[1]))
finally:
    if 'conn' in locals() or 'conn' in globals():
        conn.close()


(1, 'Data Engineer - Analyst (12 month contract Cork)', 'Cork City', 'Competitive', datetime.date(2024, 1, 5), 666, 777, 'Morgan McKinley', 'Permanent')
(2, 'Data Analyst', 'Cork', 'Competitive', datetime.date(2023, 12, 22), 666, 777, 'Morgan McKinley', 'Permanent')
(3, 'Data Analyst', 'Cork', 'Competitive', datetime.date(2024, 2, 1), 666, 777, 'Morgan McKinley', 'Permanent')
(4, 'Data Administrator', 'Cork', '€20k\u200b - €30k', datetime.date(2024, 1, 2), 666, 999, 'Morgan McKinley', 'Temporary')
(5, 'Data Administrator', 'Cork', 'Competitive', datetime.date(2024, 1, 4), 666, 999, 'Morgan McKinley', 'Temporary')
(6, 'Data Analyst', 'Cork', 'Competitive', datetime.date(2023, 12, 22), 666, 777, 'Morgan McKinley', 'Permanent')
(7, 'Data Entry Clerk', 'Cork City', 'Competitive', datetime.date(2024, 1, 3), 666, 999, 'Morgan McKinley', 'Temporary')
(8, 'Director, Consulting Data and Analytics', 'Dublin', 'Competitive', datetime.date(2024, 2, 9), 666, 777, 'Morgan McKinley', 'Permanent')
(9,